# Kleine Anfragen WordCloud

In [ ]:
#%matplotlib inline
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import psycopg2
import os
from nltk.stem.snowball import GermanStemmer

In [ ]:
os.chdir("E:\\wordclouds\\")

## Function to create the bag of words

This function takes the rows from the database and creates the bag of words (i.e. counts the occurrences).

In [ ]:
def make_bow(rows, use_stemmer=False, use_contents=False):
    
    if use_stemmer:
        stemmer = GermanStemmer()
        stemmer._GermanStemmer__step3_suffixes = () # Step 3 removes too much.
        stemmer._GermanStemmer__step1_suffixes = ('ern', 'em', 'er', 'en', 'es', 's')
    
    local_stop_words = []
    for sw in stop_words:
        if use_stemmer:
            local_stop_words.append(stemmer.stem(sw))
        else:
            local_stop_words.append(sw)
    
 
    bow = Counter([])

    for r in rows:
        txt = r[1] # Title
        
        if use_contents:
            txt += " " + r[2]
        
        for sp in stop_punct:
            txt = txt.replace(sp, " ")
        txt = txt.split()

        tokens = []
        
        
        for w in txt:
            token = w.strip()
            token = token[0].upper() + token[1:]
            if use_stemmer:
                token = stemmer.stem(token)
                
            if len(token) > 2 and token.lower() not in local_stop_words:
                illegal_prefix = False
                for sp in stop_prefixes:
                    if token.lower().startswith(sp):
                        illegal_prefix = True
                
                if not illegal_prefix:
                    tokens.append(token)
        
        bow += Counter(tokens)


    return bow

## Connect to the PostgreSQL DB

In [ ]:
#c.close()
#conn.close()
conn = psycopg2.connect("dbname=kleineanfragen user=postgres password=ayayay")
c = conn.cursor()

In [ ]:
states = ["Baden-Württemberg", "Bayern", "Berlin", "Brandenburg", "Bremen", "Hamburg", "Hessen",
          "Mecklenburg-Vorpommern", "Niedersachsen", "Nordrhein-Westfalen", "Rheinland-Pfalz",
          "Saarland", "Sachsen", "Sachsen-Anhalt", "Schleswig-Holstein", "Thüringen"]

## The stop words that will be filtered out

In [ ]:
stop_words = ["aber", "als", "am", "an", "auch", "auf", "aus", "bei", "bin", "bis", "bist", "da",
              "dadurch", "daher", "darum", "das", "daß", "dass", "dein", "deine", "dem", "den",
              "der", "des", "dessen", "deshalb", "die", "dies", "dieser", "dieses", "doch", "dort", 
              "du", "durch", "ein", "eine", "einem", "einen", "einer", "eines", "er", "es", "euer", 
              "eure", "für", "hatte", "hatten", "hattest", "hattet", "hier", "hinter", "ich", "ihr",
              "ihre", "im", "in", "ist", "ja", "jede", "jedem", "jeden", "jeder", "jedes", "jener",
              "jenes", "jetzt", "kann", "kannst", "können", "könnt", "machen", "mein", "meine",
              "mit", "muß", "mußt", "musst", "müssen", "müßt", "nach", "nachdem", "nein", "nicht", 
              "nun", "oder", "seid", "sein", "seine", "sich", "sie", "sind", "soll", "sollen",
              "sollst", "sollt", "sonst", "soweit", "sowie", "und", "unser", "unsere", "unter",
              "vom", "von", "vor", "wann", "warum", "was", "weiter", "weitere", "wenn", "wer",
              "werde", "werden", "werdet", "weshalb", "wie", "wieder", "wieso", "wir", "wird",
              "wirst", "wo", "woher", "wohin", "zu", "zum", "zur", "über", "a", "about", "above",
              "after", "again", "against", "all", "am", "an", "and", "any", "are", "aren't", "as",
              "at", "be", "because", "been", "before", "being", "below", "between", "both", "but",
              "by", "can't", "cannot", "could", "couldn't", "did", "didn't", "do", "does", "doesn't",
              "doing", "don't", "down", "during", "each", "few", "for", "from", "further", "had",
              "hadn't", "has", "hasn't", "have", "haven't", "having", "he", "he'd", "he'll", "he's",
              "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's",
              "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "isn't", "it", "it's", 
              "its", "itself", "let's", "me", "more", "most", "mustn't", "my", "myself", "no", "nor",
              "not", "of", "off", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves",
              "out", "over", "own", "same", "shan't", "she", "she'd", "she'll", "she's", "should",
              "shouldn't", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs",
              "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll",
              "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
              "very", "was", "wasn't", "we", "we'd", "we'll", "we're", "we've", "were", "weren't",
              "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's",
              "whom", "why", "why's", "with", "won't", "would", "wouldn't", "you", "you'd", "you'll",
              "you're", "you've", "your", "yours", "yourself", "yourselves"]

for s in states:
    stop_words.append(s.lower())

stop_words += ["januar", "februar", "märz", "april", "mai", "juni", "juli",
               "august", "september", "oktober", "november", "dezember"]


## Stop prefixes. Every word starting as one of these will be filtered out as well.

In [ ]:
stop_prefixes = ["bayerisch", "bremer", "bremisch", "hessisch", "mecklenburg-vorpomm", "niedersächs",
                 "nordrhein-westfäl", "nordrhein-westfal", "rheinland-pfälz", "saarländ", "sächsisch",
                 "thüring", "anfrage", "nachfrage"]

for s in states:
    stop_prefixes.append(s.lower())

## Punctuation that will be replaced by a blank.

In [ ]:
stop_punct = "!/\\()*+,./:;?[]^_`\{|\}~\"'"

## Building the query

In [ ]:
def build_query(party="all", region=False):
    # Careful, region is case sensitive!

    query = "select papers.id, title, contents, originator_type, organizations.name, bodies.name\n"
    query += " from papers\n"
    query += " inner join paper_originators on papers.id = paper_originators.paper_id\n"
    query += " inner join organizations on originator_id = organizations.id\n"
    query += " inner join bodies on papers.body_id = bodies.id\n"
    query += " where originator_type = 'Organization'\n"
    
    # If party is specified, add condition to query.
    
    if party == "cdu":
        query += " and (organizations.id = 11 or organizations.id = 22 or organizations.id = 237)"

    elif party == "spd":
        query += " and (organizations.id = 2)"
        
    elif party == "linke":
        query += " and (organizations.id = 14)"

    elif party == "afd":
        query += " and (organizations.id = 39 or organizations.id = 238)"
    
    elif party == "grüne":
        query += " and (organizations.id = 1 or organizations.id = 237)"
        
    elif party == "piraten":
        query += " and organizations.id = 13"
        
    elif party == "fdp":
        query += " and (organizations.id = 31 or organizations.id = 151)"
        
    elif party == "npd":
        query += " and (organizations.id = 30)"
        
    elif party == "fw":
        query += " and (organizations.id = 3  or organizations.id = 136)"
            
    elif party == "fraktionslos":
        query += " and (organizations.id = 15)"
        
    
        
    # Add region condition, or use all.

    if region:
        query += " and bodies.name = '{}'".format(region)
    else:
        region = "all"

    query += ";"


    return query

## Make the cloud!

In [ ]:
def create_word_cloud(party="all", region=False, use_stemmer = False, use_contents = False):
    query = build_query(party=party, region=region)
    c.execute(query)
    rows = c.fetchall()
    print("{} + {} = {} rows.".format(party, region, len(rows)))
    
    bow = make_bow(rows, use_stemmer=use_stemmer, use_contents=use_contents)
    
    if not region or region == "Bundestag":
        ger_mask = plt.imread("./region_shapes/ger.jpg")
    else:
        ger_mask = plt.imread("./region_shapes/{}.jpg".format(region))
        
    wc = WordCloud(background_color="black", mask=ger_mask, stopwords=stop_words,
                   max_words=300).generate_from_frequencies(list(bow.items()))
    

    #plt.figure(figsize=(20, 28))
    plt.figure(figsize=(30, 30))
    fig = plt.imshow(wc)
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig("./output/wordcloud_{}_{}_{}_{}.jpg".format("all" if not region else region, party, "stem" if use_stemmer else "nostem", "tc" if use_contents else "tonly"),
                bbox_inches='tight', pad_inches=0)
    
    plt.close(fig)

## Generate all the clouds!

Warning: can take some time!

In [ ]:
orgs = ["cdu", "spd", "linke", "grüne", "fw", "fraktionslos", "afd", "npd", "fdp", "piraten", "all"]
regions = states[:]
regions.append(False)
regions.append("Bundestag")

for o in orgs:
    create_word_cloud(o, False)
    
for r in regions:
    create_word_cloud("all", r)